In [67]:
%run init_script_defines.py

NameError: name 'games_trimmed' is not defined

In [ ]:
player_pcts_and_groups = build_player_pcts_and_groups()

In [98]:
def get_shots_by_player(player_id, games_trimmed = games_trimmed, player_id_games = player_id_games):
    if games_trimmed is None:
        print("Please build games_trimmed with the build_games_trimmed() function. \nPass the directory your game .json files are saved in as its argument.")
        return None
    if player_id_games is None:
        player_id_games = build_player_id_games()

    # Initialize an empty list to store the shots taken by the player.
    player_shots = []

    # Retrieve the games for the player from the player_id_games dictionary
    games = player_id_games[player_id]

    # Iterate over the keys in the games dictionary
    for year, gameIDs in games.items():
        # Iterate over the gameIDs in the list of gameIDs for the current year
        for gameID in gameIDs:
            # Retrieve the game dictionary for the current gameID
            game_dict = games_trimmed[gameID]

            # Iterate over the shots in the game
            for shot_num, shot in game_dict.items():
                periodTime = shot['about']['periodTime']
                period = shot['about']['period']

                # Check if the shot was taken in periods 1-3
                if period in range(1, 4):
                    # Check if the player took the shot
                    if shot['players'][0]['player']['id'] == player_id:
                        # Append a tuple (game_ID, shot_num) to the list of player shots
                        player_shots.append((gameID, shot_num))

    return player_shots

In [119]:
def get_player_avg_pct_and_groups(playerID, grouped_data = grouped_data):
    player_shots = get_shots_by_player(playerID)
    weighted_shots = grouped_data
    group_dict = {}
    total_ratio = 0
    total_weight = 0
    for gameID, shot in player_shots:
        x, y = get_shot_grouped_coordinates(gameID, shot)
        for _ in range(1, 850):
            weighted_shots[(x, y)].append((gameID, shot))
    for (x, y) in weighted_shots:
        group_ratio = get_goal_shot_ratio(playerID, player_shots=weighted_shots[(x, y)])
        total_ratio += group_ratio * len(weighted_shots[(x, y)])
        total_weight += len(weighted_shots[(x, y)])
        group_dict[(x, y)] = group_ratio
    return (total_ratio / total_weight, group_dict)

In [92]:
def build_player_id_games(player_teams=player_teams, team_games=team_games):
    #global player_teams
    if player_teams == None:
        player_teams = build_player_teams()
    #global team_games
    if team_games == None:
        team_games = build_team_games()

    # Initialize an empty dictionary to store the games a player has played in each year
    player_id_games = {}

    # Iterate over the keys in the player_teams dictionary
    for player_id, teams in player_teams.items():
        # Initialize an empty dictionary for the current player_id
        player_id_games[player_id] = get_single_player_games(player_id)

    # Return the player_id_games dictionary
    return player_id_games

In [101]:
def get_shot_grouped_coordinates(gameID, shotID, games_trimmed = games_trimmed):
    return (games_trimmed[gameID][shotID]['coordinates']['x']//5, games_trimmed[gameID][shotID]['coordinates']['y']//3)

In [116]:
grouped_data = build_grouped_data()

In [115]:
def build_grouped_data(coordinate_shots = coordinate_shots):
    # Initialize the new dictionary
    grouped_data = {}
    # Loop through the data points in the original dictionary
    for (x, y) in coordinate_shots.keys():
      # Use integer division to map the x and y values to the coarser grid
      x_group = x // 5
      y_group = y // 3
      
      # Use a tuple of the x and y group as the key in the new dictionary
      key = (x_group, y_group)
      
      # If the key doesn't exist in the dictionary yet, initialize it with an empty list
      if key not in grouped_data:
        grouped_data[key] = []
      
      # Append the data point to the list for this grid cell
      grouped_data[key].extend(coordinate_shots[(x,y)])
    return grouped_data

In [122]:
def get_goal_shot_ratio(player_id, player_shots=None, count=False, games_trimmed = games_trimmed):
    if games_trimmed is None:
        print("Please build games_trimmed with the build_games_trimmed() function. \nPass the directory your game .json files are saved in as its argument.")
        return None
    global player_id_games
    if player_id_games is None:
        player_id_games = build_player_id_games()

    # If player_shots is not provided, retrieve them using the get_player_shots function
    if player_shots is None:
        player_shots = get_player_shots(player_id)

    # If the player has never been on the ice for a shot, return 0
    if len(player_shots) == 0:
        return 0

    # Initialize counters for the number of goals and shots
    goal_count = 0
    shot_count = 0

    # Loop through the shot dictionaries in the player_shots list
    for shot_location in player_shots:
        game_id, shot_num = shot_location
        shot = games_trimmed[game_id][shot_num]

        # Check the event type of the shot (either 'Goal' or 'Shot')
        event_type = shot['result']['event']

        # If the event is a goal, increment the goal_count
        if event_type == 'Goal':
            goal_count += 1

        # If the event is a shot, increment the shot_count
        if event_type in ('Goal', 'Shot'):
            shot_count += 1

    # Calculate the proportion of goals to shots by dividing the number of goals by the number of shots
    proportion_of_goals = goal_count / shot_count

    if count:
        return (proportion_of_goals, shot_count)
    else:
        return (proportion_of_goals)